In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh


In [2]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE

import pandas as pd

# setup environment GEITje-7B Finetuning
# - pip install torch
# - pip install datasets
# - pip install transformers
# - pip install trl
# - pip install accelerate (restart after)
# - switch device_map='auto' to avaoid memory error

# - pip install sentencepiece
# - pip install jupyter
# - pip install protobuf 



## Notebook overview
This notebook creates predictions for the baseline models. In total, five models are tried out.
- Training function. Given a baseline model, will return scores.
- Load Data. Load all the documents, and set parameters.
- save predictions


*Previous notebook: RepairMistralPredictions*

*Next notebook: plot*

### Load file with training funcation


In [3]:
import sys
sys.path.append('../scripts/') 
import baseline as bf

### Load data

In [4]:
import pandas as pd

df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")

In [5]:
display(df.head())

,label,path,id,text,num_pages,4split,2split,MistralTokens,count_MistralTokens,LlamaTokens,count_LlamaTokens,md5_hash,balanced_split
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1874,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,1.0,train,train,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",350,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",346,2f09ba2c967bba0eecf71f846f258a78,discard
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,230,X Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,2.0,train,train,"[▁X, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",1130,"[▁X, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",1082,d14b33c32ba1e1bcff16320891bdf158,discard
2,Raadsnotulen,/home/azureuser/cloudfiles/code/blobfuse/raads...,26304,Gemeente Amsterdam\n% Gemeenteraad R\n% Raadsn...,79.0,train,train,"[▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Geme, en...",89050,"[▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Geme, en...",85359,36964ae4a84926e2f825761d980d12f4,test
3,Besluit,/home/azureuser/cloudfiles/code/blobfuse/raads...,20677,3. Interne documenten - 5271\nx Gemeente Beslu...,2.0,train,train,"[▁, 3, ., ▁Inter, ne, ▁document, en, ▁-, ▁, 5,...",1094,"[▁, 3, ., ▁Inter, ne, ▁document, en, ▁-, ▁, 5,...",1071,f2f9203231ceba0504087b493b5ffd1d,train
4,Raadsadres,/home/azureuser/cloudfiles/code/blobfuse/raads...,24174,"|\nÍ\nAmsterdam, september 2016 |\nGeachte led...",2.0,train,train,"[▁|, <0x0A>, Í, <0x0A>, Am, sterdam, ,, ▁septe...",1839,"[▁|, <0x0A>, Í, <0x0A>, Am, sterdam, ,, ▁septe...",1775,cebc20ef3921faab5a377c1a637ed22d,train


In [6]:
from collections import Counter
sys.path.append('../scripts/') 
import baseline as bf
from truncation import add_truncation_column

#set  variables, same for each model
SPLIT_COLUMN = 'balanced_split' #column that has the data split saved. must be either 2split, 4split or balanced_split. 2split = data split into train and test. 4split = data split into train, test, dev and val. 
print('Distribution of sets: ', Counter(df[SPLIT_COLUMN]))
TRAIN_SET = 'train' # must be dev or train
TEST_SET = 'test' # must be val or test
# this split column, train_set and test_set might be a bit confusing. The split_column need to have values about the split, so a row either belongs, in my case, to 'train', 'test', 'dev' or 'val'.
# Then the train_set indates which rows will be selected based on the filtering of the split column. 
# Thus if TRAIN_SET = 'train', then all rows where split_col is 'train', will be selected as the training set.
# The same goes for TEST_SET    


TEXT_COLUMN = 'text' # column where the text is
LABEL_COLUMN = 'label' # column with truth label
DATAFRAME = df.copy() # df where each row is a doc. 
FOLDER = f"{cf.output_path}/predictionsFinal/baselines" # folder where each individual prediction is saved
OVERVIEW_PATH = f"{cf.output_path}/predictionsFinal/baselines/overview.pkl" # file where score and extra data about run is saved

# needed for truncation experiment on baselines
TRUNC_COLUMN = 'trunc_txt' # column with truncated text
TOKENS_COL = 'LlamaTokens' # column with text split into tokens using model tokenizer, in this case Llama, could also be MistralTokens
THRESHOLD_COMBINATIONS =[(100,0), (200,0), (100,100)] # combinations of front and back truncation thresholds. First value in tuple is first N tokens, second value is last N tokens.

Distribution of sets:  Counter({'train': 9900, 'discard': 8718, 'test': 1100, 'val': 1100})


In [7]:
# Function to run the baseline on each truncation threshold

def run_truncation_on_baselines(baseline_function, model_name, predictions_path):
    for thresholds in THRESHOLD_COMBINATIONS:

        # select thresholds
        front_threshold = thresholds[0]
        back_threshold = thresholds[1]

        # set run_id
        run_id = f"{model_name}_first{front_threshold}_last{back_threshold}"

        # get df with truncated text column
        trunc = add_truncation_column(DATAFRAME, TEXT_COLUMN, TOKENS_COL, front_threshold,back_threshold)

        # train and get predictions
        bf.run_baseline(baseline_function, model_name, trunc, SPLIT_COLUMN, TRAIN_SET, TEST_SET, TRUNC_COLUMN, LABEL_COLUMN, predictions_path, OVERVIEW_PATH, run_id)

### Baselines

##### Baseline 1: linear SVM+tf-idf

In [8]:
from sklearn.svm import LinearSVC
model_name = 'LinearSVC'
baseline_function = LinearSVC()
run_id = f"{model_name}_fulltext"
predictions_path = f"{FOLDER}/{model_name}predictions.pkl"

print(OVERVIEW_PATH)
print(predictions_path)
linear_svm = bf.run_baseline(baseline_function, model_name , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, predictions_path, OVERVIEW_PATH, run_id)

run_truncation_on_baselines(baseline_function, model_name, predictions_path)

/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/processed_data/woo_document_classification/predictionsFinal/baselines/overview.pkl
/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/processed_data/woo_document_classification/predictionsFinal/baselines/LinearSVCpredictions.pkl


##### Baseline 2: Naive Bayes+tf-idf

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model_name = 'MultinomialNB'
baseline_function = MultinomialNB()
run_id = f"{model_name}_fulltext"
predictions_path = f"{FOLDER}/{model_name}predictions.pkl"
print(OVERVIEW_PATH)
print(predictions_path)

naive_bayes = bf.run_baseline(baseline_function, model_name , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, predictions_path, OVERVIEW_PATH, run_id)

run_truncation_on_baselines(baseline_function, model_name, predictions_path)

##### Baseline 3: Logistic Regression + tf-idf

In [ ]:
from sklearn.linear_model import LogisticRegression
model_name = 'LogisticRegression'
baseline_function = LogisticRegression()
run_id = f"{model_name}_fulltext"
predictions_path = f"{FOLDER}/{model_name}predictions.pkl"

print(OVERVIEW_PATH)
print(predictions_path)

log_reg = bf.run_baseline(baseline_function, model_name , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, predictions_path, OVERVIEW_PATH, run_id)

run_truncation_on_baselines(baseline_function, model_name, predictions_path)

##### Baseline 4: k Nearest Neigbors + tf-idf

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_name = 'KNeighborsClassifier'
baseline_function = KNeighborsClassifier()
run_id = f"{model_name}_fulltext"
predictions_path = f"{FOLDER}/{model_name}predictions.pkl"

print(OVERVIEW_PATH)
print(predictions_path)
knn = bf.run_baseline(baseline_function, model_name , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, predictions_path, OVERVIEW_PATH, run_id)

run_truncation_on_baselines(baseline_function, model_name, predictions_path)

##### Baseline 5: RandomForest + tf-idf

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_name = 'RandomForestClassifier'
baseline_function = RandomForestClassifier()
run_id = f"{model_name}_fulltext"
predictions_path = f"{FOLDER}/{model_name}predictions.pkl"

print(OVERVIEW_PATH)
print(predictions_path)

random_forest = bf.run_baseline(baseline_function, model_name , DATAFRAME, SPLIT_COLUMN, TRAIN_SET, TEST_SET,TEXT_COLUMN, LABEL_COLUMN, predictions_path, OVERVIEW_PATH, run_id)

run_truncation_on_baselines(baseline_function, model_name, predictions_path)

### Overview of all runs

In [ ]:
overview = pd.read_pickle(OVERVIEW_PATH)
display(overview)